In [1]:
import numpy as np
import faiss
import glob
import json
import matplotlib.pyplot as plt
import os
import math
import pandas as pd
import re
import time
from pathlib import Path
import sys

from langdetect import detect

In [2]:
ROOT = Path(os.getcwd()).resolve()

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))
WORK_DIR = ROOT.parents[1]

print("Main Work Directory:", WORK_DIR)

Main Work Directory: /home/toonies/Learn/training_aic


In [3]:
def process_name(name: int):
    return "0"*(6-len(str(name))) + str(name)

def sort_key(file_path):
    file_name = os.path.basename(file_path)
    match = re.match(r'(\D+)(\d+)_(V)(\d+)', file_name)
    if match:
        prefix = match.group(1)
        number = int(match.group(2))
        suffix_number = int(match.group(4))
        return (prefix, number, suffix_number)
    return (file_name, 0, 0)
  
  
def write_bin_file_ocr(bin_path: str, npy_path: str  ,method='cosine', feature_shape= 512): # Edit 512, 768
  if method in 'L2':
    index = faiss.IndexFlatL2(feature_shape)
  elif method in 'cosine':
    index = faiss.IndexFlatIP(feature_shape)
  else:
    assert f"{method} not supported"
  
  npy_files = glob.glob(os.path.join(npy_path, "*.npy"))
  npy_files_sorted = sorted(npy_files, key=sort_key)
  
  for npy_file in npy_files_sorted:
    feats = np.load(npy_file)
    index.add(feats)
    
  faiss.write_index(index, os.path.join(bin_path, f"faiss_CLIP_{method}.bin"))

  print(f'Saved {os.path.join(bin_path, f"faiss_CLIP_{method}.bin")}')
  
write_bin_file_ocr(bin_path = f"{WORK_DIR}/data/dicts/bin_clip", npy_path = f"{WORK_DIR}/data/dicts/npy_clip")


Saved /home/toonies/Learn/training_aic/data/dicts/bin_clip/faiss_CLIP_cosine.bin
